In [0]:
%py
# Definir caminhos base
# BRONZE_PATH = "/mnt/delta/bronze"
# https://github.com/neelabalan/mongodb-sample-dataset/tree/main
SOURCE_PATH = "dbfs:/Volumes/workspace/default/resource/input/mongodb-sample-dataset"
DESTINY_PATH = "/Volumes/workspace/default/delta/bronze/mongodb-sample-dataset/"

In [0]:
display(dbutils.fs.ls(f"{SOURCE_PATH}/sample_mflix"))

In [0]:
fold = f"{SOURCE_PATH}/sample_mflix/theaters.json"
df = spark.read.format('json').load(fold)

In [0]:
from pyspark.sql.functions import col

df_cinema = (
    df.withColumn("id", col("_id.$oid"))
    .withColumn("city", col("location.address.city"))
    .withColumn("state", col("location.address.state"))
    .withColumn("street1", col("location.address.street1"))
    .withColumn("street2", col("location.address.street2"))
    .withColumn("zipcode", col("location.address.zipcode"))
    .withColumn("coordinates", col("location.geo.coordinates.$numberDouble"))
    .withColumn("longitude", col("coordinates")[0]) \
    .withColumn("latitude", col("coordinates")[1]) \
    .withColumn("theaterId", col("theaterId.$numberInt")) \
    .drop("_id", "location", "theaterId", "coordinates")
)
display(df_cinema)

In [0]:
display(df)

In [0]:
sample_mflix.write.format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save(F"{DESTINY_PATH}/sample_mflix")
